# Pre. 모델 적용 규칙

I. 사용한 모델
- Linear Regression
- Ridge
- Lasso
- ElasticNet
- XGBoost
- Random Forest

II. Scaling
- 각 feature들을 정규화 적용

III. 하이퍼 파라미터 튜닝(그리드 서치 적용)
1. Ridge Regression
- alpha: 릿지 회귀의 정규화 강도. 높은 alpha 값은 모델의 복잡도를 줄이고, 과적합을 방지.
2. Lasso Regression
- alpha: 라쏘 회귀의 정규화 강도. 높은 alpha 값은 더 많은 계수를 0으로 수렴시키는 경향이 있어서 특성 선택(feature selection)에 유용.
3. ElasticNet
- alpha: 모델에서의 전체 정규화 강도. Ridge와 Lasso 회귀의 균형을 조절.
- l1_ratio: L1 정규화 (Lasso)의 비율. 0에서 1 사이의 값으로, 0에 가까울수록 Ridge 회귀와 유사하고, 1에 가까울수록 Lasso 회귀와 유사.
4. XGBoost
- n_estimators: 부스팅 반복 횟수. 모델에 포함될 트리의 개수를 결정.
- max_depth: 각 트리의 최대 깊이를 제한. 과적합을 막기 위함.
- learning_rate: 학습률. 낮은 학습률은 보다 안정적인 모델을 만들어 주지만, 더 많은 트리가 필요할 수 있음.
- subsample: 각 트리를 학습할 때 사용할 훈련 데이터의 샘플링 비율
5. Random Forest
- n_estimators: 랜덤 포레스트에서 생성할 트리의 개수
- max_depth: 각 트리의 최대 깊이를 제한
- min_samples_split: 노드를 분할하기 위해 필요한 최소 샘플 수

# 1. Required Library Import

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

# 2. Data Import

In [2]:
data = pd.read_excel("data_encoded.xlsx")

# 3. Apply Regression Model (애니메이션을 제외한 전체 데이터)

In [3]:
X = data.drop(columns=['관객수', '영화명', '개봉일'])
y = data['관객수']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grids = {
    "Linear Regression": {},
    "Ridge": {
        "alpha": [0.01, 0.1, 1, 10, 100]
    },
    "Lasso": {
        "alpha": [0.01, 0.1, 1, 10, 100]
    },
    "ElasticNet": {
        "alpha": [0.01, 0.1, 1, 10, 100],
        "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]
    },
    "XGBoost": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 7],
        "learning_rate": [0.01, 0.1, 0.2],
        "subsample": [0.7, 0.8, 0.9, 1.0]
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10]
    }
}

models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "XGBoost": XGBRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42)
}

results = {}

for name, model in models.items():
    param_grid = param_grids[name]
    
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

    if name in ["Ridge", "Lasso", "ElasticNet"]:
        grid_search.fit(X_train_scaled, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test_scaled)
    else:
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = [mae, mse, rmse, r2]

for name, metrics in results.items():
    print(f"{name} (Grid Search) - MAE: {metrics[0]:.2f}, MSE: {metrics[1]:.2f}, RMSE: {metrics[2]:.2f}, R²: {metrics[3]:.2f}")

Linear Regression (Grid Search) - MAE: 1259881.36, MSE: 4827892816714.70, RMSE: 2197246.64, R²: 0.26
Ridge (Grid Search) - MAE: 1260828.55, MSE: 4865644507504.58, RMSE: 2205820.60, R²: 0.25
Lasso (Grid Search) - MAE: 1259857.85, MSE: 4827973534582.92, RMSE: 2197265.01, R²: 0.26
ElasticNet (Grid Search) - MAE: 1274807.10, MSE: 4984851002423.98, RMSE: 2232677.99, R²: 0.24
XGBoost (Grid Search) - MAE: 1090304.54, MSE: 4153880334875.64, RMSE: 2038107.05, R²: 0.36
Random Forest (Grid Search) - MAE: 1106413.98, MSE: 4164182082994.14, RMSE: 2040632.77, R²: 0.36


## 3-1. 코로나 기간을 제외한 데이터셋 적용

In [5]:
data = pd.read_excel("data_encoded_nocorona.xlsx")

In [6]:
X = data.drop(columns=['관객수', '영화명', '개봉일'])
y = data['관객수']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grids = {
    "Linear Regression": {},
    "Ridge": {
        "alpha": [0.01, 0.1, 1, 10, 100]
    },
    "Lasso": {
        "alpha": [0.01, 0.1, 1, 10, 100]
    },
    "ElasticNet": {
        "alpha": [0.01, 0.1, 1, 10, 100],
        "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]
    },
    "XGBoost": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 7],
        "learning_rate": [0.01, 0.1, 0.2],
        "subsample": [0.7, 0.8, 0.9, 1.0]
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10]
    }
}

models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "XGBoost": XGBRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42)
}

results = {}

for name, model in models.items():
    param_grid = param_grids[name]
    
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

    if name in ["Ridge", "Lasso", "ElasticNet"]:
        grid_search.fit(X_train_scaled, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test_scaled)
    else:
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = [mae, mse, rmse, r2]

for name, metrics in results.items():
    print(f"{name} (Grid Search) - MAE: {metrics[0]:.2f}, MSE: {metrics[1]:.2f}, RMSE: {metrics[2]:.2f}, R²: {metrics[3]:.2f}")

C:\Users\J\anaconda3\envs\downgrade\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.159e+13, tolerance: 3.687e+10
  model = cd_fast.enet_coordinate_descent(


Linear Regression (Grid Search) - MAE: 354877.81, MSE: 208110727126.30, RMSE: 456191.55, R²: 0.25
Ridge (Grid Search) - MAE: 327236.23, MSE: 150154049981.15, RMSE: 387497.16, R²: 0.46
Lasso (Grid Search) - MAE: 335076.55, MSE: 197576861935.91, RMSE: 444496.19, R²: 0.29
ElasticNet (Grid Search) - MAE: 333874.19, MSE: 153476159887.88, RMSE: 391760.33, R²: 0.45
XGBoost (Grid Search) - MAE: 385003.95, MSE: 224621044388.96, RMSE: 473942.03, R²: 0.19
Random Forest (Grid Search) - MAE: 425741.81, MSE: 388587293820.39, RMSE: 623367.70, R²: -0.40


# 4. Hypothesis test

## 4-1. 언급량이 많을수록 총 관객수가 많을 것이다. (원작여부 / 시리즈여부)

In [8]:
data = pd.read_excel("data_encoded.xlsx")

In [9]:
X = data[['총 언급수', '원작유무', '시리즈']]
y = data['관객수']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grids = {
    "Linear Regression": {},
    "Ridge": {
        "alpha": [0.01, 0.1, 1, 10, 100]
    },
    "Lasso": {
        "alpha": [0.01, 0.1, 1, 10, 100]
    },
    "ElasticNet": {
        "alpha": [0.01, 0.1, 1, 10, 100],
        "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]
    },
    "XGBoost": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 7],
        "learning_rate": [0.01, 0.1, 0.2],
        "subsample": [0.7, 0.8, 0.9, 1.0]
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10]
    }
}

models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "XGBoost": XGBRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42)
}

results = {}

for name, model in models.items():
    param_grid = param_grids[name]
    
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

    if name in ["Ridge", "Lasso", "ElasticNet"]:
        grid_search.fit(X_train_scaled, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test_scaled)
    else:
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = [mae, mse, rmse, r2]

for name, metrics in results.items():
    print(f"{name} (Grid Search) - MAE: {metrics[0]:.2f}, MSE: {metrics[1]:.2f}, RMSE: {metrics[2]:.2f}, R²: {metrics[3]:.2f}")

Linear Regression (Grid Search) - MAE: 1406466.78, MSE: 6050537514405.11, RMSE: 2459784.04, R²: 0.07
Ridge (Grid Search) - MAE: 1413242.40, MSE: 6079646915019.14, RMSE: 2465694.00, R²: 0.07
Lasso (Grid Search) - MAE: 1406490.01, MSE: 6050626478327.00, RMSE: 2459802.12, R²: 0.07
ElasticNet (Grid Search) - MAE: 1436425.19, MSE: 6205942285272.30, RMSE: 2491172.87, R²: 0.05
XGBoost (Grid Search) - MAE: 1254332.60, MSE: 4894849556740.20, RMSE: 2212430.69, R²: 0.25
Random Forest (Grid Search) - MAE: 1222578.64, MSE: 4558743087310.15, RMSE: 2135121.33, R²: 0.30


## 4-2. 영화 작품성(평론가 평점)이 높을수록 총 관객수가 많을 것이다. (감독 평점)

In [11]:
X = data[['평론가 평점', '감독 평점']]
y = data['관객수']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grids = {
    "Linear Regression": {},
    "Ridge": {
        "alpha": [0.01, 0.1, 1, 10, 100]
    },
    "Lasso": {
        "alpha": [0.01, 0.1, 1, 10, 100]
    },
    "ElasticNet": {
        "alpha": [0.01, 0.1, 1, 10, 100],
        "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]
    },
    "XGBoost": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 7],
        "learning_rate": [0.01, 0.1, 0.2],
        "subsample": [0.7, 0.8, 0.9, 1.0]
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10]
    }
}

models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "XGBoost": XGBRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42)
}

results = {}

for name, model in models.items():
    param_grid = param_grids[name]
    
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

    if name in ["Ridge", "Lasso", "ElasticNet"]:
        grid_search.fit(X_train_scaled, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test_scaled)
    else:
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = [mae, mse, rmse, r2]

for name, metrics in results.items():
    print(f"{name} (Grid Search) - MAE: {metrics[0]:.2f}, MSE: {metrics[1]:.2f}, RMSE: {metrics[2]:.2f}, R²: {metrics[3]:.2f}")

Linear Regression (Grid Search) - MAE: 1446892.03, MSE: 6311985006396.34, RMSE: 2512366.42, R²: 0.03
Ridge (Grid Search) - MAE: 1453332.08, MSE: 6330486808467.46, RMSE: 2516045.87, R²: 0.03
Lasso (Grid Search) - MAE: 1446891.93, MSE: 6311984456766.53, RMSE: 2512366.31, R²: 0.03
ElasticNet (Grid Search) - MAE: 1457422.21, MSE: 6342962043053.03, RMSE: 2518523.78, R²: 0.03
XGBoost (Grid Search) - MAE: 1488634.81, MSE: 6533042005130.27, RMSE: 2555981.61, R²: -0.00
Random Forest (Grid Search) - MAE: 1465693.58, MSE: 6512717278745.32, RMSE: 2552002.60, R²: 0.00


## 4-3. 예고편 조회수가 많을수록 총 관객수가 많을 것이다. (배우 인지도)

In [13]:
X = data[['조회수', '배우 언급수']]
y = data['관객수']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grids = {
    "Linear Regression": {},
    "Ridge": {
        "alpha": [0.01, 0.1, 1, 10, 100]
    },
    "Lasso": {
        "alpha": [0.01, 0.1, 1, 10, 100]
    },
    "ElasticNet": {
        "alpha": [0.01, 0.1, 1, 10, 100],
        "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]
    },
    "XGBoost": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 7],
        "learning_rate": [0.01, 0.1, 0.2],
        "subsample": [0.7, 0.8, 0.9, 1.0]
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10]
    }
}

models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "XGBoost": XGBRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42)
}

results = {}

for name, model in models.items():
    param_grid = param_grids[name]
    
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

    if name in ["Ridge", "Lasso", "ElasticNet"]:
        grid_search.fit(X_train_scaled, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test_scaled)
    else:
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = [mae, mse, rmse, r2]

for name, metrics in results.items():
    print(f"{name} (Grid Search) - MAE: {metrics[0]:.2f}, MSE: {metrics[1]:.2f}, RMSE: {metrics[2]:.2f}, R²: {metrics[3]:.2f}")

Linear Regression (Grid Search) - MAE: 1343810.15, MSE: 5471284471840.89, RMSE: 2339077.70, R²: 0.16
Ridge (Grid Search) - MAE: 1355405.12, MSE: 5500152505670.01, RMSE: 2345240.39, R²: 0.16
Lasso (Grid Search) - MAE: 1343815.46, MSE: 5471312348186.29, RMSE: 2339083.66, R²: 0.16
ElasticNet (Grid Search) - MAE: 1350856.76, MSE: 5485474975112.44, RMSE: 2342109.09, R²: 0.16
XGBoost (Grid Search) - MAE: 1227007.98, MSE: 5147760151129.06, RMSE: 2268867.59, R²: 0.21
Random Forest (Grid Search) - MAE: 1195796.46, MSE: 4650445441775.47, RMSE: 2156489.15, R²: 0.29
